# Exercise - learning rate and batch size

1. Consider last exercise (i.e. the MNIST data). Suppose you are restricted to **training for only 2 epochs** but still want a good model. You recognize that finding the right learning rate is going to be very important. For this reason, you split your training data into a train and a validation set and use the validation set to find the optimal learning rate. Train a model with you optimized learning rate and evaluate it on your test data.
1. Recognizing that the batch size is also important for training speed, you decide to extend your above analysis to also find the optimal batch size. Once again, train a model with you optimized learning rate *and* batch size and evaluate it on your test data.
1. You have heard that momentum is important. You know that many optimizers already incorporate momentum by default, but you are now forced by your evil teacher to use SGD and otherwise repeat (1) and (2). You decide to extend your above analysis to also find the optimal momentum for SGD (see https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD for how to set it). Once again, train a model with you optimized learning rate, batch size, *and* momentum and evaluate it on your test data.

**See slides for more details!**

# Exercise 1

Consider last exercise (i.e. the MNIST data). Suppose you are restricted to **training for only 2 epochs** but still want a good model. You recognize that finding the right learning rate is going to be very important. For this reason, you split your training data into a train and a validation set and use the validation set to find the optimal learning rate. Train a model with you optimized learning rate and evaluate it on your test data.

In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# A simply way to scale data. Remember pixels are in [0, 255] so this ensures [0, 1]
x_train = x_train / 255
x_test = x_test / 255

# Split train data to also get val data
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,test_size=0.1,random_state=43)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

(54000, 28, 28) (54000,) (6000, 28, 28) (6000,) (10000, 28, 28) (10000,)


Here is (parts of) a model to get you started.

It is very helpful to wrap it inside a function since you want to call it multiple times in a loop.

Take note of the "Flatten" layer. This is important to reshape your data from (28, 28) to (784,).

Alternatively, you could reshape your data (the x's). This can be done using:

$\texttt{x = x.reshape(n, 784)}$ 

where $n$ is the number of samples (60k for training, 10k for test).

Then you don't need the Flatten layer, but remember to still specify an input shape of your first layer (i.e. 784 if you have done this reshaping).

**Note**: Do feel free to experiment with the number of layers, nodes per layer, and optimizer.

In [2]:
def build_model(learning_rate):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax'),
    ])
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'],
    )
    
    return model

Let us look at single run.

In [3]:
model = build_model(0.0001) # insert desired learning rate
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2)
model.evaluate(x_test, y_test)

Epoch 1/2
1688/1688 [==============================] - 4s 2ms/step - loss: 2.3048 - accuracy: 0.0697 - val_loss: 2.2799 - val_accuracy: 0.0792
Epoch 2/2
313/313 [==============================] - 0s 1ms/step - loss: 2.2406 - accuracy: 0.1114


[2.240609884262085, 0.11140000075101852]

Now for the optimization.

In [4]:
learning_rates = [0.1,0.01,0.001,0.00001] # must be positive floats. Default depends on optimizer

results = []

for learning_rate in learning_rates:
    model = build_model(learning_rate)
    model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2) # you may want to use verbose=0 here to not get spammed. Also remember to use epochs=2
    loss, acc = model.evaluate(x_test, y_test)
    results.append((acc, learning_rate))
    
results = pd.DataFrame(results, columns=['Accuracy', 'Learning rate'])
results

Epoch 1/2
1688/1688 [==============================] - 3s 2ms/step - loss: 0.3507 - accuracy: 0.8948 - val_loss: 0.2073 - val_accuracy: 0.9370
Epoch 2/2
313/313 [==============================] - 0s 1ms/step - loss: 0.1662 - accuracy: 0.9496
Epoch 1/2
1688/1688 [==============================] - 3s 2ms/step - loss: 0.8099 - accuracy: 0.7815 - val_loss: 0.3959 - val_accuracy: 0.8857
Epoch 2/2
313/313 [==============================] - 0s 1ms/step - loss: 0.2881 - accuracy: 0.9159
Epoch 1/2
1688/1688 [==============================] - 3s 2ms/step - loss: 2.1212 - accuracy: 0.2800 - val_loss: 1.8605 - val_accuracy: 0.5053
Epoch 2/2
313/313 [==============================] - 0s 1ms/step - loss: 1.1300 - accuracy: 0.7549
Epoch 1/2
1688/1688 [==============================] - 3s 2ms/step - loss: 2.3267 - accuracy: 0.1138 - val_loss: 2.3218 - val_accuracy: 0.1175
Epoch 2/2
313/313 [==============================] - 0s 1ms/step - loss: 2.3228 - accuracy: 0.1108


,Accuracy,Learning rate
0,0.9496,0.10000
1,0.9159,0.01000
2,0.7549,0.00100
3,0.1108,0.00001


In [5]:
results[results['Accuracy'] == results['Accuracy'].max()]

,Accuracy,Learning rate
0,0.9496,0.1


In [ ]:
# Train and evaluate final model.
# Remember to use both train and val data for training for best performance! 
# Similar to what we have done in all the other exercises/assignments

# Exercise 2

Recognizing that the batch size is also important for training speed, you decide to extend your above analysis to also find the optimal batch size. Once again, train a model with you optimized learning rate *and* batch size and evaluate it on your test data.

In [6]:
learning_rates = [0.1,0.01,0.001,0.00001] # must be positive floats. Default depends on optimizer
batch_sizes = [8,16,32,64,128] # # must be positive ints. Default is 32

results = []

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        model = build_model(learning_rate)
        model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2)# remember to pass in batch_size here! Also remember to use epochs=2
        loss, acc = model.evaluate(x_test, y_test)
        results.append((acc, learning_rate, batch_size))
    
results = pd.DataFrame(results, columns=['Accuracy', 'Learning rate', 'Batch size'])
results

Epoch 1/2
1688/1688 [==============================] - 3s 2ms/step - loss: 0.3314 - accuracy: 0.9008 - val_loss: 0.2153 - val_accuracy: 0.9350
Epoch 2/2
313/313 [==============================] - 0s 1ms/step - loss: 0.1603 - accuracy: 0.9505
Epoch 1/2
1688/1688 [==============================] - 3s 2ms/step - loss: 0.3404 - accuracy: 0.8966 - val_loss: 0.3211 - val_accuracy: 0.9052
Epoch 2/2
313/313 [==============================] - 0s 1ms/step - loss: 0.1511 - accuracy: 0.9531
Epoch 1/2
1688/1688 [==============================] - 3s 2ms/step - loss: 0.3421 - accuracy: 0.8969 - val_loss: 0.2581 - val_accuracy: 0.9200
Epoch 2/2
313/313 [==============================] - 0s 1ms/step - loss: 0.2808 - accuracy: 0.9123
Epoch 1/2
1688/1688 [==============================] - 3s 2ms/step - loss: 0.3495 - accuracy: 0.8953 - val_loss: 0.2429 - val_accuracy: 0.9215
Epoch 2/2
313/313 [==============================] - 0s 1ms/step - loss: 0.1671 - accuracy: 0.9494
Epoch 1/2
1688/1688 [===========

,Accuracy,Learning rate,Batch size
0,0.9505,0.10000,8
1,0.9531,0.10000,16
2,0.9123,0.10000,32
3,0.9494,0.10000,64
4,0.9568,0.10000,128
5,0.9190,0.01000,8
6,0.9182,0.01000,16
7,0.9147,0.01000,32
8,0.9132,0.01000,64
9,0.9179,0.01000,128


In [7]:
results[results['Accuracy'] == results['Accuracy'].max()]

,Accuracy,Learning rate,Batch size
4,0.9568,0.1,128


In [ ]:
# Train and evaluate final model.
# Remember to use both train and val data for training for best performance! 
# Similar to what we have done in all the other exercises/assignments

# Exericse 3

You have heard that momentum is important. You know that many optimizers already incorporate momentum by default, but you are now forced by your evil teacher to use SGD and otherwise repeat (1) and (2). You decide to extend your above analysis to also find the optimal momentum for SGD (see https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD for how to set it). Once again, train a model with you optimized learning rate, batch size, *and* momentum and evaluate it on your test data.

In [8]:
def build_model_with_momentum(learning_rate, momentum):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax'),
    ])
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'],
    )
    
    return model

In [10]:
from tqdm import tqdm
learning_rates = [0.1,0.01,0.001,0.00001] # must be positive floats. Default depends on optimizer
batch_sizes = [8,16,32,64,128] # # must be positive ints. Default is 32
momentums = [0.1,0.25,0.5,0.75,0.9] # must be in [0, 1). Default (for SGD) is 0.0

results = []

for learning_rate in tqdm(learning_rates):
    for batch_size in batch_sizes:
        for momentum in momentums:
            model = build_model_with_momentum(learning_rate, momentum=momentum)            
            model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2,verbose=0)# remember to pass in batch_size here! Also remember to use epochs=2
            loss, acc = model.evaluate(x_test, y_test)
            results.append((acc, learning_rate, batch_size, momentum))
    
results = pd.DataFrame(results, columns=['Accuracy', 'Learning rate', 'Batch size', 'Momentum'])
results

  0%|          | 0/4 [00:00<?, ?it/s]

313/313 [==============================] - 0s 1ms/step - loss: 0.3067 - accuracy: 0.9244


 25%|██▌       | 1/4 [02:35<07:45, 155.22s/it]

313/313 [==============================] - 1s 2ms/step - loss: 0.1512 - accuracy: 0.9526


 50%|█████     | 2/4 [05:23<05:25, 162.70s/it]

313/313 [==============================] - 1s 2ms/step - loss: 0.2822 - accuracy: 0.9166


 75%|███████▌  | 3/4 [08:24<02:51, 171.15s/it]

313/313 [==============================] - 1s 2ms/step - loss: 2.2161 - accuracy: 0.1805


100%|██████████| 4/4 [12:26<00:00, 186.72s/it]


,Accuracy,Learning rate,Batch size,Momentum
0,0.9564,0.10000,8,0.10
1,0.9371,0.10000,8,0.25
2,0.9590,0.10000,8,0.50
3,0.9465,0.10000,8,0.75
4,0.9360,0.10000,8,0.90
...,...,...,...,...
95,0.0988,0.00001,128,0.10
96,0.1227,0.00001,128,0.25
97,0.1020,0.00001,128,0.50
98,0.1174,0.00001,128,0.75


In [11]:
results[results['Accuracy'] == results['Accuracy'].max()]

,Accuracy,Learning rate,Batch size,Momentum
12,0.9614,0.10,32,0.5
44,0.9614,0.01,64,0.9


In [ ]:
# Train and evaluate final model.
# Remember to use both train and val data for training for best performance! 
# Similar to what we have done in all the other exercises/assignments